# 1. 初期設定

In [ ]:
# 環境設定
!pip install mecab-python3 # 形態素解析ライブラリ MeCab
!pip install unidic # システム辞書 UniDic
!python -m unidic download # システム辞書 UniDic

In [ ]:
# ライブラリ導入
import pandas as pd # pandas
import numpy as np # NumPy
import unicodedata # 文字コード
import MeCab # 形態素解析
from collections import Counter # カウンター
import requests # HTTP通信
import unidic # システム辞書
import re # 正規表現
import os # ファイル操作
import itertools # ループ
import ipywidgets as widgets # ウィジェット
from IPython.display import display, clear_output # 表示

from google.colab import drive # Google Drive
drive.mount('/content/drive') # Google Drive マウント

os.chdir('/content/drive/MyDrive/User') # 作業ディレクトリの変更

In [ ]:
# 教師データ
zero_mail = pd.read_csv('weight_zero.csv', encoding = 'cp932') # クレーム度 0
one_mail = pd.read_csv('weight_one.csv', encoding = 'cp932') # クレーム度 1
two_mail = pd.read_csv('weight_two.csv', encoding = 'cp932') # クレーム度 2

In [ ]:
## 処理を軽くするためにデータ数を減らしています。実装時には削除。
zero_mail = zero_mail.head(500)

In [ ]:
# クレーム度の設定（0は固定）
## この係数を変えることで「クレーム」と「クレームになる可能性があったもの」の比重を変更できます
mid = 1 # クレーム度 1の係数
high = 2 # クレーム度 2の係数

# 2. 形態素解析

In [ ]:
# 形態素解析の関数宣言 func_2-1
def morpheme_tokenizer(text_input):
  # 形態素解析の関数宣言 func_2-1-1
  def morpheme(text):
    # 形態素解析
    mecab = MeCab.Tagger()
    cell_parse = mecab.parse(text)
    lines = cell_parse.splitlines()
    # EOS（End Of Sentence）の削除
    lines = lines[:-1]
    data = []
    # 形態素解析後の各カラムの分離
    for line in lines:
      surface, feature = line.split('\t')
      feature = [None if f == '*' else f for f in feature.split(',')]
      data.append([surface, *feature])
    return pd.DataFrame(data)

  # 形態素解析の実行及びデータフレーム化
  words_df = morpheme(text_input) # func_2-1-1

  # 表層形・品詞・原形のみ抽出
  if not words_df.empty:
    surface_parts_df = words_df.iloc[:, 0:2].reset_index(drop=True)
    original = words_df.iloc[:, 8:9].reset_index(drop=True)
    words_df_edit = pd.concat([surface_parts_df, original], axis=1)
    COLUMNS = ['表層形', '品詞', '原形']
    words_df_edit.columns = COLUMNS
    # 名詞・形容詞・動詞・副詞のみ抽出
    words_df_edit = words_df_edit[words_df_edit['品詞'].isin(['名詞', '形容詞', '動詞', '副詞'])]
    # 形態素解析結果をリスト化
    word_cloud_list = list(words_df_edit['原形'].dropna())
    # ひらがなのみ除外（ひらがなのみもカウントする場合はコメントアウト）
    def remove_eng(word): # func_2-1-2
        return re.sub(r'-[a-zA-Z]+', '', word)
    kana_re = re.compile("[^あ-ゖ]")
    word_cloud_list_edit = [s for s in word_cloud_list if kana_re.match(s)]
    word_cloud_list_edit = [remove_eng(t) for t in word_cloud_list_edit] # func_2-1-2
    return word_cloud_list_edit
  else:
    return []

# 3. コンビネーション

In [ ]:
# コロケーションカウントの関数宣言 func_3-1
def count_collocation(text):
  cells = [morpheme_tokenizer(text)] # func_2-1
  cells_combs = [list(itertools.combinations(cell,2)) for cell in cells]
  words_combs = [[tuple(sorted(words)) for words in cell] for cell in cells_combs]
  target_combs = []
  for words_comb in words_combs:
    target_combs.extend(words_comb)
  # 一時変数に格納
  temp_count = Counter(target_combs)
  temp_df = pd.DataFrame([{"前" : i[0][0], "後": i[0][1], "Count":i[1]} for i in temp_count.most_common()])
  return temp_df

In [ ]:
# コロケーション合算（一時変数を結合）の関数宣言 func_3-2
def add_count(combs_df, temp_count):
  combs_df = pd.merge(combs_df, temp_count, on=['前', '後'], how='outer', suffixes=('1', '2'))
  combs_df.fillna(0, inplace=True)
  combs_df['Count'] = combs_df[['Count1', 'Count2']].sum(axis=1)
  combs_df = combs_df[['前', '後', 'Count']]
  combs_df.sort_values('Count', ascending=False, inplace=True)
  return combs_df

In [ ]:
# 問い合わせのリストをループ（形態素解析・コンビネーション）func_3-3
def loop_df(mail_df):
  combs_df = pd.DataFrame(columns=['前', '後', 'Count'])
  for cell in mail_df['受付内容']:
    combs_df = add_count(combs_df, count_collocation(cell)) # func_3-1, func_3-2
  return combs_df

In [ ]:
# 教師データへの実行
zero_combs = loop_df(zero_mail) # func_3-3
one_combs = loop_df(one_mail) # func_3-3
two_combs = loop_df(two_mail) # func_3-3
dict_list = [zero_combs, one_combs, two_combs] # データフレームをリスト化

# 4. 比率計算

In [ ]:
# 比率計算の関数宣言 func_4-1
def calc_rate():
  # 比率計算用データフレームの作成 func_4-1-1
  def create_rate_df(zero_df, claim_df):
    rate_df = pd.merge(zero_df, claim_df, on = ['前', '後'], how = 'inner')
    rate_df.columns = ['前', '後', 'Zero Count', 'Claim Count']
    rate_df['All Count'] = rate_df['Zero Count'] + rate_df['Claim Count']
    return rate_df
  # 比率計算の関数宣言 func_4-1-2
  def calc_rate(rate_df):
    rate_df['Rate'] = rate_df['Claim Count'] / rate_df['All Count']
    rate_df['Rate'] = rate_df['Rate']
    rate_df = rate_df.sort_values(['Rate', 'All Count'], ascending = False).reset_index(drop = True)
    return rate_df
  # データフレーム作成の実行
  zero_one = create_rate_df(dict_list[0], dict_list[1]) # func_4-1-1
  zero_two = create_rate_df(dict_list[0], dict_list[2]) # func_4-1-1
  claim_dict_list = [zero_one, zero_two] # データフレームをリスト化
  # 比率計算の実行
  one_rate = calc_rate(claim_dict_list[0]) # func_4-1-2
  two_rate = calc_rate(claim_dict_list[1]) # func_4-1-2
  rate_dict_list = [one_rate, two_rate]
  # クレーム度によるの重みづけ
  rate_dict_list[0]['Raw Score'] = mid # クレーム度 1の係数
  rate_dict_list[1]['Raw Score'] = high # クレーム度 2の係数
  # 重みづけしたデータフレームの結合
  red_rate = pd.concat([rate_dict_list[0], rate_dict_list[1]])
  red_rate.drop_duplicates(subset = ['前', '後'], keep = 'last', inplace = True)
  red_rate.reset_index(drop = True, inplace = True)
  # スコア算出
  red_rate['Score'] = red_rate['Raw Score'] * red_rate['Rate']
  red_rate.reset_index(drop = True, inplace = True)
  return red_rate

In [ ]:
# スコア表の作成
red_rate = calc_rate() # func_4-1

# 5. スコア計算

In [ ]:
# スコア計算の関数宣言 func_5-1
def calc_score(mail):
  global new_count
  # 不要字削除の関数宣言 1 func_5-1-1
  def replace_words(text, word_to_remove, word_to_replace):
    if isinstance(text, str):
      return text.replace(word_to_remove, word_to_replace)
    else:
      return text

  # 不要字削除の関数宣言 2 func_5-1-2
  def remove(word):
    if isinstance(word, str):
      word = re.sub(r'[ 　_＿(（[［【「『<＜]', '', word)
      word = re.sub(r'[)）\]］】」』>＞]', '', word)
      word = re.sub(r'\d+\.*\d*', '', word)
      word = re.sub(r'[①②③④⑤⑥⑦⑧⑨⑩〇?？!！&＆%％*＊+＋=＝:：■◆●★・/\／→⇒←、。,，.．※#×～↓↑]', '', word)
      word = re.sub(r'[-]', '', word)
      return word
    else:
      return word

  # データクレンジング実行
  word_to_remove = '\n'
  word_to_replace = ''

  mail = remove(mail) # func_5-1-1
  mail = replace_words(mail, word_to_remove, word_to_replace) # func_5-1-2

  # 新メールの形態素解析
  new_count = count_collocation(mail) # func_3-1

  # スコア計算
  new_red_count = pd.merge(red_rate, new_count, on=['前', '後'], how='inner') # スコア表との比較
  mail_claim_score = (new_red_count['Count'] * new_red_count['Score']).sum() # スコア計算
  max_score = sum(new_count['Count'] * high) # スコア最大値計算 # クレーム度 2の係数
  mail_score = (max_score - mail_claim_score) / max_score # スコア正規化（割合）
  mail_score = int(mail_score * 10000) # スコア正規化（0-10000）
  return mail_score

In [ ]:
# リスト化された新規問い合わせの一括スコア計算
new_mail_df = pd.read_csv('weight_two.csv', encoding = 'cp932')
new_mail_df['スコア'] = new_mail_df['受付内容'].apply(calc_score) # func_5-1
new_mail_df.sort_values('スコア', ascending=True, inplace=True)
new_mail_df.reset_index(drop=True, inplace=True)
new_mail_df

# 6. 学習

In [ ]:
# 評価関数宣言 func_6-1
def add_evaluation_dropdown(options_list):
  global new_mail_df
  evaluation_column = [widgets.Dropdown(options=options_list, description='') for _ in range(len(new_mail_df))] #ドロップダウン
  new_mail_df['評価'] = evaluation_column
  return new_mail_df

options_list = [('クレームになる可能性がない', 0), ('クレームになる可能性がある', 1), ('クレームである', 2)]

# 新規問い合わせリストの全行評価 func_6-2
def display_dataframe_with_widgets():
  global new_mail_df
  add_evaluation_dropdown(options_list) # func_6-1
  for index, row in new_mail_df.iterrows():
    hbox = widgets.HBox([widgets.Label(value=str(index)), row['評価']])
    display(hbox) # 表示

# 評価別分類 func_6-3
def slice_new_mail_df(new_mail_df):
  new_zero_df = new_mail_df[new_mail_df['評価'].apply(lambda x: x.value) == 0]
  new_one_df = new_mail_df[new_mail_df['評価'].apply(lambda x: x.value) == 1]
  new_two_df = new_mail_df[new_mail_df['評価'].apply(lambda x: x.value) == 2]
  return new_zero_df, new_one_df, new_two_df

# 問い合わせのリストをループ（形態素解析・コンビネーション） func_6-4
def new_combs(new_zero_df, new_one_df, new_two_df):
  new_zero_combs = loop_df(new_zero_df) # func_3-3
  new_one_combs = loop_df(new_one_df) # func_3-3
  new_two_combs = loop_df(new_two_df) # func_3-3
  return new_zero_combs, new_one_combs, new_two_combs

# データの加算 func_6-5
def add_data(new_zero_combs, new_one_combs, new_two_combs):
  global dict_list, red_rate
  dict_list[0] = add_count(dict_list[0], new_zero_combs) # func_3-2
  dict_list[1] = add_count(dict_list[1], new_one_combs) # func_3-2
  dict_list[2] = add_count(dict_list[2], new_two_combs) # func_3-2

  dict_list[0].sort_values(by='Count', ascending=False, inplace=True)
  dict_list[1].sort_values(by='Count', ascending=False, inplace=True)
  dict_list[2].sort_values(by='Count', ascending=False, inplace=True)

  red_rate = calc_rate()

# 登録ボタンの作成
button = widgets.Button(
  description='登録',
  disabled=False,
  button_style='',
)

# ボタンクリック時の処理 func_6-6
def on_button_click(b):
  print(new_mail_df)
  new_zero_df, new_one_df, new_two_df = slice_new_mail_df(new_mail_df) # func_6-3
  new_zero_combs, new_one_combs, new_two_combs = new_combs(new_zero_df, new_one_df, new_two_df) # func_6-4
  add_data(new_zero_combs, new_one_combs, new_two_combs) # func_6-5
  clear_output()
  print('登録完了')

button.on_click(on_button_click) # func_6-6

In [ ]:
display_dataframe_with_widgets() # func_6-2
display(button)